# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
pip install cartopy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install geoviews

   ---------------------------------------- 0.0/547.3 kB ? eta -:--:--
   ---------------------------------------- 547.3/547.3 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
from cartopy import crs
import json

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,5.32,51,25,3.13,US,1743521024
1,1,adamstown,-25.0660,-130.1015,24.78,81,1,3.53,PN,1743521026
2,2,nalut,30.3333,10.8500,23.87,14,1,9.52,LY,1743521028
3,3,papatowai,-46.5619,169.4708,12.25,89,100,4.63,NZ,1743521029
4,4,caleta de carquin,-11.0925,-77.6267,21.83,81,97,3.37,PE,1743521031


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

city_map = city_data_df.hvplot.points('Lng', 'Lat', 
                           geo= True,
                           tiles= 'OSM',
                           size= 'Humidity',
                           color='City', 
                           alpha=0.5,
                           frame_width = 900,
                           frame_height = 500,
                          )
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[(city_data_df['Max Temp'] > 27) & 
                            (city_data_df['Max Temp'] < 29) & 
                            (city_data_df['Wind Speed'] < 15) &
                            (city_data_df['Cloudiness'] < 15 )].copy()

# Drop any rows with null values
filtered_df.dropna(how='all', inplace=True)

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,puerto ayora,-0.7393,-90.3518,28.08,90,11,0.45,EC,1743520789
96,96,konarka,19.9000,86.1167,27.12,78,1,7.10,IN,1743521153
115,115,karachi,24.9056,67.0822,27.90,54,0,1.03,PK,1743521092
170,170,gopalpur,19.2667,84.9167,27.10,77,3,8.17,IN,1743521252
187,187,bela,25.9333,81.9833,28.32,8,0,3.25,IN,1743521275
193,193,toliara,-23.3500,43.6667,27.14,74,0,3.64,MG,1743521282
254,254,hithadhoo,-0.6000,73.0833,28.33,74,6,2.05,MV,1743521370
261,261,chilmari,25.5559,89.6715,28.80,30,4,1.87,BD,1743521379
331,331,chuhar jamali,24.3895,67.9923,28.97,31,0,4.12,PK,1743521476
359,359,morondava,-20.2833,44.2833,28.53,77,2,4.26,MG,1743521516


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City','Country','Lat', 'Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,puerto ayora,EC,-0.7393,-90.3518,90,
96,konarka,IN,19.9000,86.1167,78,
115,karachi,PK,24.9056,67.0822,54,
170,gopalpur,IN,19.2667,84.9167,77,
187,bela,IN,25.9333,81.9833,8,
193,toliara,MG,-23.3500,43.6667,74,
254,hithadhoo,MV,-0.6000,73.0833,74,
261,chilmari,BD,25.5559,89.6715,30,
331,chuhar jamali,PK,24.3895,67.9923,31,
359,morondava,MG,-20.2833,44.2833,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row.iloc[2]
    Lng = row.iloc[3]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
konarka - nearest hotel: Hotel Sun Village
karachi - nearest hotel: Quetta Agha Hotel
gopalpur - nearest hotel: Swosti
bela - nearest hotel: Hotel City Palace
toliara - nearest hotel: Ambary
hithadhoo - nearest hotel: Pebbles Inn
chilmari - nearest hotel: No hotel found
chuhar jamali - nearest hotel: No hotel found
morondava - nearest hotel: Hotel Sharon Menabe
nizwa - nearest hotel: Nizwa Heritage Inn
ciudad victoria - nearest hotel: Central


,City,Country,Lat,Lng,Humidity,Hotel Name
27,puerto ayora,EC,-0.7393,-90.3518,90,Hostal La Mirada De Solitario George
96,konarka,IN,19.9000,86.1167,78,Hotel Sun Village
115,karachi,PK,24.9056,67.0822,54,Quetta Agha Hotel
170,gopalpur,IN,19.2667,84.9167,77,Swosti
187,bela,IN,25.9333,81.9833,8,Hotel City Palace
193,toliara,MG,-23.3500,43.6667,74,Ambary
254,hithadhoo,MV,-0.6000,73.0833,74,Pebbles Inn
261,chilmari,BD,25.5559,89.6715,30,No hotel found
331,chuhar jamali,PK,24.3895,67.9923,31,No hotel found
359,morondava,MG,-20.2833,44.2833,77,Hotel Sharon Menabe


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points('Lng', 'Lat', 
                                   geo= True,
                                   tiles= 'OSM',
                                   size= 200,          
                                   color='City',      
                                   alpha=0.6,
                                   frame_width = 900,
                                   frame_height = 500,
                                   hover_cols=['Hotel Name', 'Country'],
                                   legend='right'
                        
                          )
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)